In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt



###UP IS ORIGIN

import math
import torch.optim as optim


### SEC 3
from torchvision.utils import save_image






# custom dataloader for .npy file
class numpyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = torch.from_numpy(data).float()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x
    
    def __len__(self):
        return len(self.data)


In [2]:
### Setting up Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Below codes activates when want to use cpu
#device = torch.device('cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce GTX 1050
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
transform       = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                            ])

print("Load traindata..")
# for training

BATCH_SIZE = 400

traindata       = np.load('train.npy')
traindataset    = numpyDataset(traindata, transform)
trainloader     = DataLoader(traindataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6)

print("Train data is ready")


Load traindata..
Train data is ready


In [ ]:
size_img = 120 * 80
size_layer_1 = 100
size_layer_2 = 50
size_layer_3 = 12
size_layer_4 = 3

class autoencoder(nn.Module):

    def __init__(self) :
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential (
            nn.Conv2d(1, 6, kernel_size = 5),
            nn.ReLU(True),
            nn.Conv2d(6, 16,kernel_size= 7),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16,6,kernel_size= 7),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,1,kernel_size= 5),
            nn.ReLU(True))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    

    
max_epochs = 10000 # For prevent system down

#5 # 6
learning_rate = 1e-4

In [ ]:
model = autoencoder().cuda()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)
criterion = nn.MSELoss()

epoch_history = []
epoch_loss_history = []
epoch_loss_std_history = []
exit_flag = 0

for epoch in range(max_epochs):
    # =========== Saving Origin imgs =========== #
    if epoch == 0 :
        for data in enumerate(traindata) :
            img = torch.tensor(data[1]).cuda()
            to_img = transforms.ToPILImage()
            pic_origin = to_img(img.view(120,80).cpu())
            pic_origin.save('./mlp_img/{}_origin.jpg'.format(data[0]))

    # ================= Train ===================
    loss_logs = []
    for data in enumerate(trainloader):
        
        order = data[0]
        img = data[1].clone().detach()
   
        img = img.cuda()
     
        # Create Noisy img
        
        # Random sigma between 0.01 - 0.04
        noise_level = torch.tensor(0.01) + torch.rand(1) * 0.03
        normal_distrib = torch.distributions.Normal(torch.tensor([0.0]),noise_level)
        noise = normal_distrib.sample(img.shape).view(img.shape).cuda()
        
        noisy_img = img + noise
        
        # ===================forward=====================
        output = model(noisy_img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        loss_logs.append(loss.item())
        optimizer.step()
        
        
        # ================= This code for Debug ====================#
        if epoch == 1 :
            if order == 2 :
                to_img = transforms.ToPILImage()
                pic_origin = to_img(img[2].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(21))
                
                pic_origin = to_img(img[6].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(25))
            else :
                to_img = transforms.ToPILImage()
                pic_origin = to_img(img[3].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(42))
            
        '''        
        if order == 2 or order == 4 :
            if order == 2 :
                to_img = transforms.ToPILImage()
                
                ## 21
                pic_input = to_img(noisy_img[2].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(21,epoch))
                 
                pic_output = to_img(output[2].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(21,epoch))

                ## 25
                pic_input = to_img(noisy_img[6].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(25,epoch))
                
                
                pic_output = to_img(output[6].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(25,epoch))
            else :
                ## 42
                to_img = transforms.ToPILImage()
                pic_input = to_img(noisy_img[3].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(42,epoch))
                
                
                pic_output = to_img(output[3].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(42,epoch))
        '''
    epoch_loss = np.mean(loss_logs)
    epoch_loss_std = np.std(loss_logs)
        
    # ===================log========================
    print('[epoch {}], loss: (training) {:.10f}'
          .format(epoch, epoch_loss))
    
    
    # ================ loss diff check ============
    epoch_loss_history.append(epoch_loss)
    epoch_loss_std_history.append(epoch_loss_std)
    epoch_history.append(epoch)
    
    if epoch != 0 :
        loss_diff = abs(epoch_loss_history[epoch] - epoch_loss_history[epoch-1])
        if loss_diff <= 0.0000001 and epoch_loss_history[epoch-1] < 0.0004 :
            exit_flag += 1
            if exit_flag >= 2 :
                break
        else :
            exit_flag = 0
        

[epoch 0], loss: (training) 0.0812775181
[epoch 1], loss: (training) 0.0452465953
[epoch 2], loss: (training) 0.0245372997
[epoch 3], loss: (training) 0.0236828827
[epoch 4], loss: (training) 0.0204302626
[epoch 5], loss: (training) 0.0185520342
[epoch 6], loss: (training) 0.0170450663
[epoch 7], loss: (training) 0.0159968739
[epoch 8], loss: (training) 0.0153909969
[epoch 9], loss: (training) 0.0149522916
[epoch 10], loss: (training) 0.0146222755
[epoch 11], loss: (training) 0.0142893741
[epoch 12], loss: (training) 0.0139314397
[epoch 13], loss: (training) 0.0134784920
[epoch 14], loss: (training) 0.0128903853
[epoch 15], loss: (training) 0.0121857982
[epoch 16], loss: (training) 0.0113727538
[epoch 17], loss: (training) 0.0105438710
[epoch 18], loss: (training) 0.0096954034
[epoch 19], loss: (training) 0.0088409942
[epoch 20], loss: (training) 0.0080654986
[epoch 21], loss: (training) 0.0073952800
[epoch 22], loss: (training) 0.0068516213
[epoch 23], loss: (training) 0.0064223711
[e

[epoch 194], loss: (training) 0.0012071895
[epoch 195], loss: (training) 0.0012822095
[epoch 196], loss: (training) 0.0011797082
[epoch 197], loss: (training) 0.0012287714
[epoch 198], loss: (training) 0.0012421312
[epoch 199], loss: (training) 0.0012093307
[epoch 200], loss: (training) 0.0012448231
[epoch 201], loss: (training) 0.0012448345
[epoch 202], loss: (training) 0.0011829492
[epoch 203], loss: (training) 0.0011822172
[epoch 204], loss: (training) 0.0012156147
[epoch 205], loss: (training) 0.0012020230
[epoch 206], loss: (training) 0.0011708694
[epoch 207], loss: (training) 0.0012294894
[epoch 208], loss: (training) 0.0011874187
[epoch 209], loss: (training) 0.0011388006
[epoch 210], loss: (training) 0.0012175571
[epoch 211], loss: (training) 0.0012089299
[epoch 212], loss: (training) 0.0011855576
[epoch 213], loss: (training) 0.0012279924
[epoch 214], loss: (training) 0.0011579908
[epoch 215], loss: (training) 0.0011558153
[epoch 216], loss: (training) 0.0011661501
[epoch 217]

[epoch 385], loss: (training) 0.0008904924
[epoch 386], loss: (training) 0.0008711912
[epoch 387], loss: (training) 0.0008530539
[epoch 388], loss: (training) 0.0008359215
[epoch 389], loss: (training) 0.0008720332
[epoch 390], loss: (training) 0.0008228734
[epoch 391], loss: (training) 0.0008775338
[epoch 392], loss: (training) 0.0008908237
[epoch 393], loss: (training) 0.0009124482
[epoch 394], loss: (training) 0.0008691049
[epoch 395], loss: (training) 0.0008372843
[epoch 396], loss: (training) 0.0008334005
[epoch 397], loss: (training) 0.0008605251
[epoch 398], loss: (training) 0.0009020896
[epoch 399], loss: (training) 0.0007944158
[epoch 400], loss: (training) 0.0009078754
[epoch 401], loss: (training) 0.0008538873
[epoch 402], loss: (training) 0.0008375146
[epoch 403], loss: (training) 0.0008651862
[epoch 404], loss: (training) 0.0008488204
[epoch 405], loss: (training) 0.0008517959
[epoch 406], loss: (training) 0.0008011678
[epoch 407], loss: (training) 0.0007926536
[epoch 408]

[epoch 576], loss: (training) 0.0008480817
[epoch 577], loss: (training) 0.0006895232
[epoch 578], loss: (training) 0.0007360394
[epoch 579], loss: (training) 0.0006910415
[epoch 580], loss: (training) 0.0007456006
[epoch 581], loss: (training) 0.0007786349
[epoch 582], loss: (training) 0.0008189619
[epoch 583], loss: (training) 0.0007567649
[epoch 584], loss: (training) 0.0007781892
[epoch 585], loss: (training) 0.0007798583
[epoch 586], loss: (training) 0.0007408966
[epoch 587], loss: (training) 0.0007453264
[epoch 588], loss: (training) 0.0007435267
[epoch 589], loss: (training) 0.0007779620
[epoch 590], loss: (training) 0.0007088333
[epoch 591], loss: (training) 0.0007426530
[epoch 592], loss: (training) 0.0006857366
[epoch 593], loss: (training) 0.0007353961
[epoch 594], loss: (training) 0.0007551078
[epoch 595], loss: (training) 0.0007647913
[epoch 596], loss: (training) 0.0007274751
[epoch 597], loss: (training) 0.0007278506
[epoch 598], loss: (training) 0.0007509862
[epoch 599]

[epoch 767], loss: (training) 0.0006916292
[epoch 768], loss: (training) 0.0006979530
[epoch 769], loss: (training) 0.0006307718
[epoch 770], loss: (training) 0.0006581339
[epoch 771], loss: (training) 0.0006438193
[epoch 772], loss: (training) 0.0006431155
[epoch 773], loss: (training) 0.0006977333
[epoch 774], loss: (training) 0.0006745931
[epoch 775], loss: (training) 0.0007001676
[epoch 776], loss: (training) 0.0006729120
[epoch 777], loss: (training) 0.0006733925
[epoch 778], loss: (training) 0.0006176190
[epoch 779], loss: (training) 0.0007091640
[epoch 780], loss: (training) 0.0006676903
[epoch 781], loss: (training) 0.0006852561
[epoch 782], loss: (training) 0.0006492578
[epoch 783], loss: (training) 0.0006832538
[epoch 784], loss: (training) 0.0006616059
[epoch 785], loss: (training) 0.0006489185
[epoch 786], loss: (training) 0.0006978939
[epoch 787], loss: (training) 0.0007488489
[epoch 788], loss: (training) 0.0006684614
[epoch 789], loss: (training) 0.0006680490
[epoch 790]

...

[epoch 1146], loss: (training) 0.0005808757
[epoch 1147], loss: (training) 0.0006195216
[epoch 1148], loss: (training) 0.0006352827
[epoch 1149], loss: (training) 0.0006080360
[epoch 1150], loss: (training) 0.0006367767
[epoch 1151], loss: (training) 0.0006754626
[epoch 1152], loss: (training) 0.0006382072
[epoch 1153], loss: (training) 0.0006316203
[epoch 1154], loss: (training) 0.0006144314
[epoch 1155], loss: (training) 0.0006260450
[epoch 1156], loss: (training) 0.0006238874
[epoch 1157], loss: (training) 0.0006399257
[epoch 1158], loss: (training) 0.0005920902
[epoch 1159], loss: (training) 0.0006659311
[epoch 1160], loss: (training) 0.0005667122
[epoch 1161], loss: (training) 0.0006054286
[epoch 1162], loss: (training) 0.0006148076
[epoch 1163], loss: (training) 0.0006061580
[epoch 1164], loss: (training) 0.0005964229
[epoch 1165], loss: (training) 0.0005965693
[epoch 1166], loss: (training) 0.0005830857
[epoch 1167], loss: (training) 0.0005922119
[epoch 1168], loss: (training) 0

[epoch 1333], loss: (training) 0.0005516177
[epoch 1334], loss: (training) 0.0005806662
[epoch 1335], loss: (training) 0.0006089850
[epoch 1336], loss: (training) 0.0005902378
[epoch 1337], loss: (training) 0.0005905240
[epoch 1338], loss: (training) 0.0005490595
[epoch 1339], loss: (training) 0.0006135632
[epoch 1340], loss: (training) 0.0005949451
[epoch 1341], loss: (training) 0.0005540959
[epoch 1342], loss: (training) 0.0006039690
[epoch 1343], loss: (training) 0.0005791854
[epoch 1344], loss: (training) 0.0005896618
[epoch 1345], loss: (training) 0.0005970491
[epoch 1346], loss: (training) 0.0005855722
[epoch 1347], loss: (training) 0.0005513154
[epoch 1348], loss: (training) 0.0005482925
[epoch 1349], loss: (training) 0.0005652932
[epoch 1350], loss: (training) 0.0005483094
[epoch 1351], loss: (training) 0.0005979709
[epoch 1352], loss: (training) 0.0005750240
[epoch 1353], loss: (training) 0.0005307622
[epoch 1354], loss: (training) 0.0006017479
[epoch 1355], loss: (training) 0

[epoch 1520], loss: (training) 0.0005547520
[epoch 1521], loss: (training) 0.0005717801
[epoch 1522], loss: (training) 0.0005312403
[epoch 1523], loss: (training) 0.0005491513
[epoch 1524], loss: (training) 0.0005602044
[epoch 1525], loss: (training) 0.0005869220
[epoch 1526], loss: (training) 0.0005240847
[epoch 1527], loss: (training) 0.0005959957
[epoch 1528], loss: (training) 0.0005857943
[epoch 1529], loss: (training) 0.0005256570
[epoch 1530], loss: (training) 0.0005851190
[epoch 1531], loss: (training) 0.0006011777
[epoch 1532], loss: (training) 0.0006532176
[epoch 1533], loss: (training) 0.0005888274
[epoch 1534], loss: (training) 0.0004908328
[epoch 1535], loss: (training) 0.0006070290
[epoch 1536], loss: (training) 0.0005914759
[epoch 1537], loss: (training) 0.0005276151
[epoch 1538], loss: (training) 0.0005934738
[epoch 1539], loss: (training) 0.0005467969
[epoch 1540], loss: (training) 0.0005079780
[epoch 1541], loss: (training) 0.0005934675
[epoch 1542], loss: (training) 0

[epoch 1707], loss: (training) 0.0005924658
[epoch 1708], loss: (training) 0.0005623395
[epoch 1709], loss: (training) 0.0005111965
[epoch 1710], loss: (training) 0.0005778897
[epoch 1711], loss: (training) 0.0005749702
[epoch 1712], loss: (training) 0.0006260757
[epoch 1713], loss: (training) 0.0005115580
[epoch 1714], loss: (training) 0.0005631219
[epoch 1715], loss: (training) 0.0005153292
[epoch 1716], loss: (training) 0.0005640069
[epoch 1717], loss: (training) 0.0004795134
[epoch 1718], loss: (training) 0.0005259080
[epoch 1719], loss: (training) 0.0005555720
[epoch 1720], loss: (training) 0.0005782927
[epoch 1721], loss: (training) 0.0005223815
[epoch 1722], loss: (training) 0.0006143300
[epoch 1723], loss: (training) 0.0005717799
[epoch 1724], loss: (training) 0.0005431652
[epoch 1725], loss: (training) 0.0005600836
[epoch 1726], loss: (training) 0.0004996088
[epoch 1727], loss: (training) 0.0005679121
[epoch 1728], loss: (training) 0.0005215751
[epoch 1729], loss: (training) 0

[epoch 1894], loss: (training) 0.0005013875
[epoch 1895], loss: (training) 0.0006522260
[epoch 1896], loss: (training) 0.0005552744
[epoch 1897], loss: (training) 0.0005767188
[epoch 1898], loss: (training) 0.0005414270
[epoch 1899], loss: (training) 0.0005752080
[epoch 1900], loss: (training) 0.0006113672
[epoch 1901], loss: (training) 0.0005267336
[epoch 1902], loss: (training) 0.0005456202
[epoch 1903], loss: (training) 0.0005862948
[epoch 1904], loss: (training) 0.0006235691
[epoch 1905], loss: (training) 0.0004919769
[epoch 1906], loss: (training) 0.0005562024
[epoch 1907], loss: (training) 0.0006261494
[epoch 1908], loss: (training) 0.0005768343
[epoch 1909], loss: (training) 0.0005246896
[epoch 1910], loss: (training) 0.0005766314
[epoch 1911], loss: (training) 0.0005439453
[epoch 1912], loss: (training) 0.0005402007
[epoch 1913], loss: (training) 0.0005402902
[epoch 1914], loss: (training) 0.0005518040
[epoch 1915], loss: (training) 0.0005971618
[epoch 1916], loss: (training) 0

In [ ]:
print(int(torch.randint(4,(1,1))))

In [ ]:
test = []
for data in enumerate(trainloader):
        
    order = data[0]
    img = data[1].clone().detach()

    img = img.cuda()

    # Create Noisy img

    # Random sigma between 0.01 - 0.04
    
    noise_level = torch.tensor(0.01) + torch.rand(1) * 0.03
    normal_distrib = torch.distributions.Normal(torch.tensor([0.0]),noise_level)
    noise = normal_distrib.sample(img.shape).view(img.shape).cuda()

    noisy_img = img + noise

    # ===================forward=====================
    output = model(noisy_img)
    loss = criterion(output, img)
    # ===================backward====================
    optimizer.zero_grad()
    test.append(loss.item())
test = np.array(test)
test_mean = test.mean()
print("Test loss : {:10f}".format(test_mean))

test = []

for data in enumerate(trainloader):
        
    order = data[0]
    img = data[1].clone().detach()

    img = img.cuda()

    # Create Noisy img

    # Random sigma between 0.01 - 0.04
    
    noise_level = torch.tensor(0.01) + int(torch.randint(4,(1,1)))* 0.01
    normal_distrib = torch.distributions.Normal(torch.tensor([0.0]),noise_level)
    noise = normal_distrib.sample(img.shape).view(img.shape).cuda()

    noisy_img = img + noise

    # ===================forward=====================
    output = model(noisy_img)
    loss = criterion(output, img)
    # ===================backward====================
    optimizer.zero_grad()
    test.append(loss.item())

test = np.array(test)
test_mean = test.mean()
print("Test loss : {:10f}".format(test_mean))


In [ ]:

graph_x = []
graph_y = []

graph = plt.plot(epoch_history,epoch_loss_history, color = 'orange', label = 'Train Loss')
plt.title("Loss")
plt.legend(['Train Loss'])
plt.show()


graph_x = []
graph_y = []

graph = plt.plot(epoch_history,epoch_loss_std_history, color = 'red', label = 'Train Loss')
plt.title("Std")
plt.legend(['Train Loss Std'])
plt.show()


In [ ]:


# for testing
testdata        = np.load('test.npy')
testdataset     = numpyDataset(testdata, transform)
testloader      = DataLoader(testdataset, batch_size=1, shuffle=False, num_workers=2)

result_for_submit = None    # this is for submit file

for batch_idx, data in enumerate(testloader):

    result_of_test = model(data.cuda())
    
    #print(result_of_test)
    
    if batch_idx == 0:
        result_for_submit = result_of_test
    else:
        try:
            result_for_submit = torch.cat([result_for_submit, result_of_test], dim=0)

        except RuntimeError:
            transposed = torch.transpose(result_of_test, 2, 3)
            result_for_submit = torch.cat([result_for_submit, transposed], dim=0)

            
       
result_for_submit = result_for_submit.cpu()
     


# the submit_file.shape must be (400,1,120,80) 
submit_file = result_for_submit.detach().numpy()
print("submit_file.shape :",submit_file.shape)

np.save('jesoonkang20170937_u2.npy', submit_file)
print("Test Result Save Success")

